In [ ]:
import pandas as pd

In [ ]:
brfss = pd.read_csv("./temp/edit_brfss.csv")
checkup = pd.read_csv("./temp/건강검진_221227.csv", encoding="cp949")
regionheart = pd.read_csv("./temp/지역별 심장질환.csv", encoding="cp949")
medical = pd.read_csv("./temp/hos_all.csv")

In [ ]:
def check_dataframe(df):
    print(df.head())
    print(df.info())
    
    for col in df.columns:
        if len(df[col].unique()) <= 10:
            print(df[col].value_counts())
            print("*"*20)

## brfss
1. CHOLESTEROL, LIVER1, LIVER2 열 제거
2. SMOKE를 제외한 열 중 하나라도 결측값이 있는 행 제거
3. HEIGHT, WEIGHT 열 5 단위로 변경

In [ ]:
check_dataframe(brfss)

   HEARTDISEASE SEX  HEIGHT     WEIGHT  BLIND  DEAF  DIABETE  CHOLESTEROL  \
0           0.0   F  149.86  32.658651    0.0   0.0      0.0          1.0   
1           0.0   F  167.64        NaN    1.0   0.0      1.0          1.0   
2           0.0   F  165.10  77.110703    0.0   0.0      1.0          2.0   
3           0.0   F  162.56  88.450512    0.0   0.0      1.0          1.0   
4           1.0   M  180.34  93.440028    0.0   1.0      1.0          1.0   

   HYPERTENSION  KIDNEY  LIVER1  LIVER2  ALCOHOL  SMOKE  
0           0.0     0.0     NaN     NaN      0.0    1.0  
1           1.0     1.0     NaN     NaN      0.0    0.0  
2           1.0     0.0     NaN     NaN      0.0    0.0  
3           1.0     0.0     NaN     NaN      1.0    0.0  
4           0.0     0.0     NaN     NaN      0.0    0.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438693 entries, 0 to 438692
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        -------------- 

### 열 제거
- CHOLESTEROL, LIVER 제거

In [ ]:
brfss.drop(["CHOLESTEROL", "LIVER1", "LIVER2"], axis=1, inplace=True)

In [ ]:
brfss["SEX"].replace({"M": 1, "F": 0}, inplace=True)

### 결측치 처리
- ALCOHOL 제외한 열 중에서 하나라도 결측치가 있는 행 제거

In [ ]:
brfss.isnull().sum()

HEARTDISEASE     2655
SEX                 0
HEIGHT          24559
WEIGHT          40575
BLIND           17351
DEAF            16398
DIABETE           985
HYPERTENSION     1912
KIDNEY           1813
ALCOHOL         30589
SMOKE           24461
dtype: int64

In [ ]:
brfss.isnull().sum() / len(brfss)

HEARTDISEASE    0.006052
SEX             0.000000
HEIGHT          0.055982
WEIGHT          0.092491
BLIND           0.039552
DEAF            0.037379
DIABETE         0.002245
HYPERTENSION    0.004358
KIDNEY          0.004133
ALCOHOL         0.069728
SMOKE           0.055759
dtype: float64

In [ ]:
col_list = list(brfss.columns)
col_list.remove("ALCOHOL")

brfss.dropna(how="any", subset=col_list, inplace=True)

In [ ]:
brfss["ALCOHOL"].fillna(9, inplace=True)

### HEIGHT, WEIGHT
- 검진자의 키(5CM 단위)
    - 예) 100-104CM --> 100CM
- 검진자의 몸무게(5KG 단위)
    - 예) 25~29KG -> 25KG

In [ ]:
brfss["HEIGHT"] // 5 * 5

0         145.0
2         165.0
3         160.0
4         180.0
5         190.0
          ...  
438687    180.0
438688    155.0
438690    180.0
438691    180.0
438692    160.0
Name: HEIGHT, Length: 378646, dtype: float64

In [ ]:
brfss["HEIGHT"] = brfss["HEIGHT"] // 5 * 5

In [ ]:
brfss["WEIGHT"] = brfss["WEIGHT"] // 5 * 5

In [ ]:
brfss.drop(["HEIGHT", "WEIGHT"], axis=1, inplace=True)

### 확인 및 저장

In [ ]:
check_dataframe(brfss)

   HEARTDISEASE  SEX  BLIND  DEAF  DIABETE  HYPERTENSION  KIDNEY  ALCOHOL  \
0           0.0    0    0.0   0.0      0.0           0.0     0.0      0.0   
2           0.0    0    0.0   0.0      1.0           1.0     0.0      0.0   
3           0.0    0    0.0   0.0      1.0           1.0     0.0      1.0   
4           1.0    1    0.0   1.0      1.0           0.0     0.0      0.0   
5           0.0    1    0.0   0.0      0.0           0.0     0.0      0.0   

   SMOKE  
0    1.0  
2    0.0  
3    0.0  
4    0.0  
5    1.0  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 378646 entries, 0 to 438692
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   HEARTDISEASE  378646 non-null  float64
 1   SEX           378646 non-null  int64  
 2   BLIND         378646 non-null  float64
 3   DEAF          378646 non-null  float64
 4   DIABETE       378646 non-null  float64
 5   HYPERTENSION  378646 non-null  float64
 6   KIDN

In [ ]:
brfss.to_csv("../data/brfss.csv", index=False)

## 건강검진
1. LIVER 열 제거
2. SEX 열 F, M으로 변경
3. 지역 열 긴 이름으로 변경(서울 -> 서울특별시)
4. encoding 변경(cp949 -> utf-8)

In [ ]:
check_dataframe(checkup)

   지역  SEX  HEIGHT  WEIGHT  BLIND  DEAF  DIABETE  HYPERTENSION  KIDNEY  LIVER  \
0  세종    1     165      60      0     0        0             0       0      0   
1  대구    2     150      65      0     0        0             0       0      0   
2  서울    2     155      55      0     0        0             0       0      0   
3  울산    1     160      70      0     1        1             0       0      1   
4  경기    2     155      50      0     0        0             1       0      0   

   ALCOHOL  SMOKE  
0      0.0      0  
1      0.0      0  
2      0.0      0  
3      0.0      0  
4      0.0      0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2039244 entries, 0 to 2039243
Data columns (total 12 columns):
 #   Column        Dtype  
---  ------        -----  
 0   지역            object 
 1   SEX           int64  
 2   HEIGHT        int64  
 3   WEIGHT        int64  
 4   BLIND         int64  
 5   DEAF          int64  
 6   DIABETE       int64  
 7   HYPERTENSION  int64  
 8   KIDNE

In [ ]:
# LIVER 제거
checkup.drop(["LIVER", "HEIGHT", "WEIGHT"], axis=1, inplace=True)

In [ ]:
region = {"강원": "강원도", "경기": "경기도", "경남": "경상남도", "경북": "경상북도", 
          "전남": "전라남도", "전북": "전라북도", "충남": "충청남도", "충북": "충청북도", 
          "광주": "광주광역시", "대구": "대구광역시", "대전": "대전광역시", "부산": "부산광역시", 
          "서울": "서울특별시", "울산": "울산광역시", "인천": "인천광역시", 
          "세종": "세종특별자치시", "제주": "제주특별자치도"}

checkup["SEX"].replace({1: 1, 2: 0}, inplace=True)
checkup["지역"].replace(region, inplace=True)

In [ ]:
checkup.isnull().sum()

지역              0
SEX             0
BLIND           0
DEAF            0
DIABETE         0
HYPERTENSION    0
KIDNEY          0
ALCOHOL         0
SMOKE           0
dtype: int64

In [ ]:
check_dataframe(checkup)

        지역  SEX  BLIND  DEAF  DIABETE  HYPERTENSION  KIDNEY  ALCOHOL  SMOKE
0  세종특별자치시    1      0     0        0             0       0      0.0      0
1    대구광역시    0      0     0        0             0       0      0.0      0
2    서울특별시    0      0     0        0             0       0      0.0      0
3    울산광역시    1      0     1        1             0       0      0.0      0
4      경기도    0      0     0        0             1       0      0.0      0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2039244 entries, 0 to 2039243
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   지역            object 
 1   SEX           int64  
 2   BLIND         int64  
 3   DEAF          int64  
 4   DIABETE       int64  
 5   HYPERTENSION  int64  
 6   KIDNEY        int64  
 7   ALCOHOL       float64
 8   SMOKE         int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 140.0+ MB
None
1    1054132
0     985112
Name: SEX, dtype: int64
****************

In [ ]:
checkup.to_csv("../data/checkup_221227.csv", index=False, encoding="utf-8")

## 지역별 심장환자 현황
1. encoding 변경(cp949 -> utf-8)

In [ ]:
check_dataframe(regionheart)

      지역  심장질환사망자  심장장애      주민등록인구
0    강원도   4900.0   121   1538492.0
1    경기도  25708.0  1295  13565450.0
2   경상남도  10312.0   268   3314183.0
3   경상북도   9880.0   311   2626609.0
4  광주광역시   2712.0   140   1441611.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   지역       17 non-null     object 
 1   심장질환사망자  17 non-null     float64
 2   심장장애     17 non-null     int64  
 3   주민등록인구   17 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 672.0+ bytes
None


In [ ]:
regionheart.isnull().sum()

지역         0
심장질환사망자    0
심장장애       0
주민등록인구     0
dtype: int64

In [ ]:
regionheart.columns

Index(['지역', '심장질환사망자', '심장장애', '주민등록인구'], dtype='object')

In [ ]:
regionheart = regionheart[['지역', '주민등록인구', '심장질환사망자', '심장장애']]

In [ ]:
regionheart.to_csv("./temp/region_heart.csv", index=False, encoding="utf-8")

## 의료수준
1. 첫 번째 열 삭제
2. 일반의, 전문의, 간호사, 임상병리사 열 데이터 타입 변경(object -> int)
3. 지역 열 긴 이름으로 변경

In [ ]:
def change_to_int(x):
    return x.str.replace(",","").replace("-", "0").astype("int64")

In [ ]:
hospital = pd.read_excel("./rawdata/1_지역별종별요양기관수현황.xls")
df = hospital[["시도", "상급종합병원", "종합병원", "병원"]]

In [ ]:
heart_hospital = pd.read_excel("./rawdata/1_지역별의원표시과목별현황.xls")
heart_hospital.drop([0], axis=0, inplace=True)
df = pd.merge(left=df, right=heart_hospital[["시도", "심장혈관흉부외과"]], on="시도", how="outer")

In [ ]:
doctor = pd.read_excel("./rawdata/1_지역별종별의료인력현황1.xls")
df = pd.merge(left=df, right=doctor[["시도", "일반의", "전문의"]], on="시도", how="outer")

In [ ]:
nurse = pd.read_excel("./rawdata/1_지역별종별의료인력현황2.xls")
df = pd.merge(left=df, right=nurse[["시도", "간호사", "임상병리사"]], on="시도", how="outer")

In [ ]:
heart_doctor = pd.read_excel("./rawdata/1_지역별종별전문과목별전문의현황.xls")
df = pd.merge(left=df, right=heart_doctor[["시도", "심장혈관흉부외과"]], on="시도", how="outer")
df.rename({"심장혈관흉부외과_x": "심장혈관흉부외과의원", "심장혈관흉부외과_y": "심장혈관흉부외과전문의"}, axis=1, inplace=True)

In [ ]:
room = pd.read_excel("./rawdata/1_지역별종별입원실현황.xls", skiprows=1)
room = room[['Unnamed: 0', '구분', '일반입원실(상급)', '일반입원실(상급).1', '일반입원실(일반)',
             '일반입원실(일반).1', '중환자실_신생아', '중환자실_신생아.1', '중환자실_성인',
             '중환자실_성인.1', '중환자실_소아', '중환자실_소아.1']]

# 구분
room = room[(room["구분"] == "상급종합병원") | (room["구분"] == "종합병원") | (room["구분"] == "병원") | (room["구분"] == "요양기관종별")]
room.drop(["구분"], axis=1, inplace=True)
room.drop([0], axis=0, inplace=True)
room.reset_index(drop=True, inplace=True)

col_list = list(room.columns)[1:]
room[col_list] = room[col_list].apply(change_to_int, axis=1)

# groupby
temp = room.groupby("Unnamed: 0").sum()
temp["입원병상"] = temp["일반입원실(상급)"] + temp["일반입원실(일반)"]
temp["입원병실"] = temp["일반입원실(상급).1"] + temp["일반입원실(일반).1"]
temp["중환병상"] = temp["중환자실_신생아"] + temp["중환자실_성인"] + temp["중환자실_소아"]
temp["중환병실"] = temp["중환자실_신생아.1"] + temp["중환자실_성인.1"] + temp["중환자실_소아.1"]

room_final = temp[["입원병상", "입원병실", "중환병상", "중환병실"]]
room_final.reset_index(inplace=True)
room_final.rename({"Unnamed: 0": "시도"}, axis=1, inplace=True)
df = pd.merge(left=df, right=room_final, on="시도", how="outer")

C:\Users\ParkMinjeong\AppData\Local\Temp\ipykernel_22108\2978542024.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_final.rename({"Unnamed: 0": "시도"}, axis=1, inplace=True)


In [ ]:
medical = df.copy()

medical.rename({"시도": "지역"}, axis=1, inplace=True)
medical[["일반의", "전문의", "간호사", "임상병리사"]] = medical[["일반의", "전문의", "간호사", "임상병리사"]].apply(change_to_int, axis=1)

region = {"강원": "강원도", "경기": "경기도", "경남": "경상남도", "경북": "경상북도", 
          "전남": "전라남도", "전북": "전라북도", "충남": "충청남도", "충북": "충청북도", 
          "광주": "광주광역시", "대구": "대구광역시", "대전": "대전광역시", "부산": "부산광역시", 
          "서울": "서울특별시", "울산": "울산광역시", "인천": "인천광역시", 
          "세종": "세종특별자치시", "제주": "제주특별자치도"}

medical["지역"].replace(region, inplace=True)

In [ ]:
df = pd.read_excel("./rawdata/119_구급대_현황_20221220173712.xls", skiprows=1)

temp = df[["본부별(1)", "운영구급차 (대)", "구급대원 (명)"]].drop([0, 1, 2, 21, 22], axis=0)

# 타입 변환
temp[["운영구급차 (대)", "구급대원 (명)"]] = temp[["운영구급차 (대)", "구급대원 (명)"]].astype("int64")

# 컬럼명 변환
temp.columns = ["지역", "구급차", "구급대원"]

# reset index
temp.reset_index(drop=True, inplace=True)

# 창원 + 경남 -> 경남
temp.iloc[15, 1:] = temp.iloc[15, 1:] + temp.iloc[17, 1:]
temp.drop([17], axis=0, inplace=True)

# 제주도 -> 제주특별자치도
temp["지역"].replace({"제주도": "제주특별자치도"}, inplace=True)

In [ ]:
medical = pd.merge(left=medical, right=temp, on="지역")

In [ ]:
medical.fillna(0, inplace=True)

### 확인 및 저장

In [ ]:
check_dataframe(medical)

      지역  상급종합병원  종합병원   병원  심장혈관흉부외과의원   일반의    전문의    간호사  임상병리사  \
0  서울특별시      14    42  222        10.0  1287  25109  58475   6831   
1  부산광역시       3    25  136         8.0   377   6966  19705   1982   
2  인천광역시       3    17   56         1.0   260   4433  12551   1343   
3  대구광역시       5    13   88         3.0   184   5060  13948   1578   
4  광주광역시       2    21   84         0.0   148   3092   9414    941   

   심장혈관흉부외과전문의   입원병상   입원병실  중환병상  중환병실  구급차  구급대원  
0          305  13846  45730   250  2660  177  1473  
1           83   7646  24605    64   860   70   633  
2           55   3249  12248    47   610   72   592  
3           51   4580  16622    60   584   55   495  
4           33   4427  13894    27   408   30   270  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 0 to 16
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   지역           17 non-null     object 
 1   상급종합병원       17 non-n

In [ ]:
medical.to_csv("./temp/medical.csv", index=False, encoding="utf-8")

## 지역별 심장질환 + 의료수준

In [ ]:
region = pd.merge(left=regionheart, right=medical, on="지역")

In [ ]:
check_dataframe(region)

      지역      주민등록인구  심장질환사망자  심장장애  상급종합병원  종합병원   병원  심장혈관흉부외과의원   일반의  \
0    강원도   1538492.0   4900.0   121       2    13   37         1.0   220   
1    경기도  13565450.0  25708.0  1295       5    62  277         9.0  1039   
2   경상남도   3314183.0  10312.0   268       3    23  134         2.0   420   
3   경상북도   2626609.0   9880.0   311       0    20   55         2.0   404   
4  광주광역시   1441611.0   2712.0   140       2    21   84         0.0   148   

     전문의    간호사  임상병리사  심장혈관흉부외과전문의   입원병상   입원병실  중환병상  중환병실  구급차  구급대원  
0   2211   6622    661           36   2398   8403    53   438  128  1014  
1  19686  46768   5526          209  13890  48092   226  2209  261  1896  
2   4856  14959   1829           72   6636  22556    50   555  139  1194  
3   3167  10143    988           51   3458  11404    49   495  146  1279  
4   3092   9414    941           33   4427  13894    27   408   30   270  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 0 to 16
Data columns (total 19 

In [ ]:
region.to_csv("../data/region.csv", index=False, encoding="utf-8")

In [ ]:
region_dict = {"강원": "강원도", "경기": "경기도", "경남": "경상남도", "경북": "경상북도", 
          "전남": "전라남도", "전북": "전라북도", "충남": "충청남도", "충북": "충청북도", 
          "광주": "광주광역시", "대구": "대구광역시", "대전": "대전광역시", "부산": "부산광역시", 
          "서울": "서울특별시", "울산": "울산광역시", "인천": "인천광역시", 
          "세종": "세종특별자치시", "제주": "제주특별자치도"}
region_reverse = dict()
for k, v in region_dict.items():
    region_reverse[v] = k
region_reverse
region["지역"] = region.지역.replace(region_reverse)
region.to_csv("../data/region_v.csv", index=False)